In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/chest_xray/chest_xray"))

# Any results you write to the current directory are saved as output.

['test', 'val', '.DS_Store', 'train']


In [2]:
import tensorflow.keras
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_set = train_datagen.flow_from_directory(
        '../input/chest_xray/chest_xray/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 5216 images belonging to 2 classes.


In [5]:
test_set = test_datagen.flow_from_directory(
        '../input/chest_xray/chest_xray/test',
        target_size=(64, 64),
        batch_size=32
    ,
        class_mode='binary')


Found 624 images belonging to 2 classes.


In [6]:
classifier=Sequential()

In [7]:
classifier.add(Conv2D(filters=64, kernel_size=(3,3) , strides=(1,1) ,padding='valid', input_shape=(64,64,3),activation='relu'))

In [8]:
classifier.add(MaxPooling2D(pool_size = (2, 2),strides=(1,1)))

In [9]:
classifier.add(Conv2D(filters=128, kernel_size=(1,1) , strides=(1,1) ,padding='valid',activation='relu'))

In [10]:
classifier.add(MaxPooling2D(pool_size = (4, 4),strides=(2,2)))

In [11]:
classifier.add(Conv2D(filters=256, kernel_size=(3,3) , strides=(2,2) ,padding='valid', input_shape=(64,64,3),activation='relu'))

In [12]:
classifier.add(MaxPooling2D(pool_size = (3, 3),strides=(1,1)))

In [13]:
classifier.add(Flatten())

In [14]:
classifier.add(Dense(units=256, activation='relu' ))
classifier.add(Dense(units=128, activation='relu' ))
classifier.add(Dense(units=1, activation='sigmoid' ))

In [15]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 128)       8320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
__________

In [17]:
classifier.fit_generator(
        train_set,
        steps_per_epoch=5216,
        epochs=1,
        validation_data=test_set,
        validation_steps=624)

Epoch 1/1
5216/5216 [==============================] - 2387s 458ms/step - loss: 0.1430 - acc: 0.9455 - val_loss: 0.2536 - val_acc: 0.9068


In [18]:
classifier.save('medical.h5')